In [116]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

ELECTRIC_DIR = os.path.join(os.path.abspath("./historical_data"), "electric_data")
DATA_FILE = os.path.join(ELECTRIC_DIR, "psco.json")

df: pd.DataFrame = pd.read_json(DATA_FILE, typ="frame", orient="records", convert_dates=["dates"])
df.set_index("date", inplace=True)

In [117]:
## See what would get cut off using various inter-quartile range metrics
# calculate inter-quartile range
iqr = (df.quantile(0.75, numeric_only=False) - df.quantile(0.25, numeric_only=False))['demand']

median = df.median(numeric_only=True)['demand']

# Find values within `iqr_mult` multiples of the median
iqr_mult = 3
min_demand = median - (iqr * iqr_mult)
max_demand = median + (iqr * iqr_mult)
print(f"inter-quartile range: {iqr}, median: {median}, valid data range: {min_demand} - {max_demand}")
df['abs_diff'] = abs(df['demand'] - median)
df['iqr_mult'] = df['abs_diff'] / iqr

inter-quartile range: 1117.0, median: 5093.0, valid data range: 1742.0 - 8444.0


In [118]:
# Look at the biggest outliers
df.sort_values("abs_diff", ascending=False).head(20)

,demand,abs_diff,iqr_mult
date,,,
2015-07-02 06:00:00,60576.0,55483.0,49.671441
2016-08-09 18:00:00,11499.0,6406.0,5.735004
2018-02-02 07:00:00,0.0,5093.0,4.559534
2019-03-10 10:00:00,0.0,5093.0,4.559534
2016-12-06 07:00:00,186.0,4907.0,4.393017
2021-07-29 00:00:00,9853.0,4760.0,4.261415
2020-03-08 07:00:00,370.0,4723.0,4.228290
2021-07-28 23:00:00,9814.0,4721.0,4.226500
2019-07-19 23:00:00,9640.0,4547.0,4.070725


In [119]:
## Kind of a weird metric here, but it seems to work
good_criterion = df['demand'].map(lambda d: d > 1000 and d < 11000)
bad_criterion = ~good_criterion

# Print invalid rows
df[bad_criterion]

,demand,abs_diff,iqr_mult
date,,,
2015-07-02 06:00:00,60576.0,55483.0,49.671441
2015-10-31 09:00:00,NaN,NaN,NaN
2015-11-01 00:00:00,NaN,NaN,NaN
2015-11-01 09:00:00,NaN,NaN,NaN
2015-11-02 00:00:00,NaN,NaN,NaN
2016-03-14 01:00:00,NaN,NaN,NaN
2016-08-09 18:00:00,11499.0,6406.0,5.735004
2016-12-06 07:00:00,186.0,4907.0,4.393017
2018-02-02 07:00:00,0.0,5093.0,4.559534


In [120]:
# Print valid rows
df[good_criterion]


,demand,abs_diff,iqr_mult
date,,,
2015-07-01 07:00:00,4875.0,218.0,0.195166
2015-07-01 08:00:00,4618.0,475.0,0.425246
2015-07-01 09:00:00,4427.0,666.0,0.596240
2015-07-01 10:00:00,4344.0,749.0,0.670546
2015-07-01 11:00:00,4374.0,719.0,0.643688
...,...,...,...
2022-11-06 20:00:00,4296.0,797.0,0.713518
2022-11-06 21:00:00,4435.0,658.0,0.589078
2022-11-06 22:00:00,4483.0,610.0,0.546106


In [121]:
# replace outliers with nan
df = df.where(good_criterion, np.nan)

# spot check
df.loc["2015-07-02 06:00:00"]

demand     NaN
abs_diff   NaN
iqr_mult   NaN
Name: 2015-07-02 06:00:00, dtype: float64

In [122]:
# interpolate missing values
df.interpolate(inplace=True)

In [123]:
# re-spot check
df.loc["2015-07-02 06:00:00"]


demand      5125.000000
abs_diff     467.000000
iqr_mult       0.418084
Name: 2015-07-02 06:00:00, dtype: float64

In [115]:
# spot check 2
df.loc['2022-07-19']

,demand,abs_diff,iqr_mult
date,,,
2022-07-19 00:00:00,9103.0,4010.0,3.589973
2022-07-19 01:00:00,8888.0,3795.0,3.397493
2022-07-19 02:00:00,8630.0,3537.0,3.166517
2022-07-19 03:00:00,8337.0,3244.0,2.904208
2022-07-19 04:00:00,7948.0,2855.0,2.555953
2022-07-19 05:00:00,7230.0,2137.0,1.913160
2022-07-19 06:00:00,6595.0,1502.0,1.344673
2022-07-19 07:00:00,6109.0,1016.0,0.909579
2022-07-19 08:00:00,5896.0,803.0,0.718890


In [124]:
tmp_grouped = df.groupby(lambda x: x.date, sort=False, as_index=True)
grouped = tmp_grouped.agg(
    daily_demand=("demand", np.sum),
    num_hours_reported=("demand", np.count_nonzero)
)

grouped.reset_index(inplace=True)
grouped[["index"]] = grouped[["index"]].astype(np.datetime64)
grouped.set_index("index", inplace=True)

daily_demand          177786.0
num_hours_reported        24.0
Name: 2022-07-19 00:00:00, dtype: float64

In [125]:
# Drop days with less than 24 hours of data (usually first & last day of range)
grouped = grouped[grouped.num_hours_reported == 24].drop(labels="num_hours_reported", axis=1)

In [126]:
grouped_file_path = os.path.join(ELECTRIC_DIR, "psco-daily-dataframe_test.json")
grouped.to_json(grouped_file_path)